In [ ]:
# Copyright (c) 2023, Extension and contributors
# For license information, please see license.txt

import frappe


def execute(filters=None):
    columns = get_columns()
    data = get_data(filters)
    conditions = get_conditions(filters)
    return columns, data

def get_columns():
    columns = [
        {
            "label": "Bill No.",
            "fieldname": "name",
            "fieldtype": "Data",
            "width": 150
        },
        {
            "label": "Item Code",
            "fieldname": "item_code",
            "fieldtype": "Data",
            "width": 150
        },
        {
            "label": "Item Name",
            "fieldname": "item_name",
            "fieldtype": "Data",
            "width": 150
        },
        {
            "label": "Date",
            "fieldname": "transaction_date",
            "fieldtype": "Date",
            "width": 150
        },
         {
            "label": "Party Code",
            "fieldname": "supplier",
            "fieldtype": "Data",
            "width": 150
        },
        {
            "label": "Party Name",
            "fieldname": "supplier_name",
            "fieldtype": "Data",
            "width": 150
        },
        {
            "label": "Branch",
            "fieldname": "cost_center",
            "fieldtype": "Data",
            "width": 150
        },
        {
            "label": "Warehouse",
            "fieldname": "warehouse",
            "fieldtype": "Data",
            "width": 150
        },
      
        {
            "label": "Order_Qty",
            "fieldname": "Order_Qty",
            "fieldtype": "Int",
            "width": 150
        },
        {
            "label": "Received_Qty",
            "fieldname": "Received_Qty",
            "fieldtype": "Int",
            "width": 150
        },
        {
            "label": "Pending_Qty",
            "fieldname": "Pending_Qty",
            "fieldtype": "Int",
            "width": 150
        },
    ]
    return columns

def get_data(filters):
    condition = get_conditions(filters)
   
    data = frappe.db.sql(f"""
        SELECT 
            T1.name,
            T1.item_code,
            T1.item_name,
            T1.transaction_date,
            T1.cost_center,
            T1.supplier,
            T1.supplier_name,
            T2.warehouse,
            T1.Order_Qty,
            COALESCE(T2.Received_Qty, 0) AS Received_Qty,
            T1.Order_Qty - COALESCE(T2.Received_Qty, 0) AS Pending_Qty
        FROM
            (
                SELECT 
                    PO.name,
                    POI.item_code,
                    POI.item_name,
                    PO.transaction_date,
                    PO.cost_center,
                    PO.supplier,
                    PO.supplier_name,
                    SUM(POI.qty) AS Order_Qty
                FROM 
                    `tabPurchase Order` AS PO
                LEFT JOIN 
                    `tabPurchase Order Item` AS POI ON PO.name = POI.parent
                WHERE 
                    PO.type = "Closed" AND PO.status != "Cancelled"
                GROUP BY 
                    PO.name, POI.item_code
            ) AS T1
        LEFT JOIN
            (
                SELECT 
                    PRI.purchase_order_,
                    PRI.item_code,
                    PRI.warehouse,
                    SUM(PRI.received_qty) AS Received_Qty
                FROM 
                    `tabPurchase Receipt` AS PR
                LEFT JOIN 
                    `tabPurchase Receipt Item` AS PRI ON PRI.parent = PR.name
                WHERE 
                    PR.is_return != 1  AND PR.status != "Cancelled"  
                GROUP BY 
                    PRI.purchase_order_, PRI.item_code
            ) AS T2
        ON T1.name = T2.purchase_order_ 
        where %s
        
        """%condition, as_dict=True)

    return data



def get_conditions(filters):
    conditions = ""
    if filters.get("from_date"):
        conditions += "T1.transaction_date >= '%s'" % filters["from_date"]
    if filters.get("to_date"):
        conditions += "and T1.transaction_date <= '%s'" % filters["to_date"]
    if filters.get("cost_center"):
        conditions += "and T1.cost_center = '%s'" % filters["cost_center"]
    if filters.get("supplier"):
        conditions += "and T1.supplier = '%s'" % filters["supplier"]
        
    return conditions

    